## 1: Setup

In [1]:
a=5

In [2]:
import sys
import os
import json
import pandas as pd

# Add project root
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "..")))

print("✅ Evaluation Tools Loaded")

✅ Evaluation Tools Loaded


## 2: Run Detector Eval

In [3]:
# Execute the detector evaluation script
# This runs the Agent on the Synthetic Data and compares to Gold Labels
!python eval_detector.py --data ../data/test_labels/synthetic_sales.parquet --gold ../data/test_labels/anomalies_gold.jsonl


Evaluation: Loading data from ../data/test_labels/synthetic_sales.parquet
Agent found 3594 anomalies.
Gold dataset has 2 labeled anomalies.

Final Score:
{
  "metric": "Detector Quality",
  "true_positives": 2,
  "false_positives": 3151,
  "missed_labels": 0,
  "precision": 0.0006,
  "recall": 1.0,
  "f1_score": 0.0013
}


2025-11-26 19:39:50,744 - agents.anomaly_stats_agent - INFO - Running Global Z-Score Detector on Sales (w=30, t=2.0)
2025-11-26 19:39:50,757 - agents.anomaly_stats_agent - INFO - Running Grouped IQR Detector on Region (w=14, k=1.5)
2025-11-26 19:39:50,826 - agents.anomaly_stats_agent - INFO - Running Grouped IQR Detector on Category (w=14, k=1.5)
2025-11-26 19:39:50,861 - agents.anomaly_stats_agent - INFO - Running Percentage Drop Detector on Category (threshold=5.0%)
2025-11-26 19:39:50,919 - agents.anomaly_stats_agent - INFO - Running Percentage Spike Detector on Category (threshold=50.0%)


## 3: Run Schema Eval

In [4]:
# Find the latest orchestration run to check LLM quality
import glob
runs = sorted(glob.glob("../outputs/runs/run_*"))
if runs:
    latest_enriched = f"{runs[-1]}/enriched_anomalies.json"
    print(f"Evaluating Schema for: {latest_enriched}")
    
    !python eval_schema_compliance.py --file "{latest_enriched}"
else:
    print("❌ No runs found to evaluate.")

Evaluating Schema for: ../outputs/runs\run_20251126T133741Z_d9f6e3/enriched_anomalies.json
🔍 Checking schema compliance for: ../outputs/runs\run_20251126T133741Z_d9f6e3/enriched_anomalies.json
{
  "metric": "LLM Schema Compliance",
  "total_records": 5,
  "valid_records": 5,
  "compliance_rate": 1.0,
  "errors": []
}


## 4: Show Report Card

In [5]:
# Load results
try:
    with open("results_detector.json", "r") as f:
        det = json.load(f)
    with open("results_schema.json", "r") as f:
        sch = json.load(f)

    print(
        f"""
    ╔════════════════════════════════════╗
    ║   SALESOPS AGENT REPORT CARD       ║
    ╠════════════════════════════════════╣
    ║ 1. DETECTOR (Math)                 ║
    ║    Recall (Found):      {det['recall']*100:>5.1f}%      ║
    ║    Precision (Noise):   {det['precision']*100:>5.1f}%      ║
    ║    F1 Score:            {det['f1_score']:>5.2f}       ║
    ╠════════════════════════════════════╣
    ║ 2. EXPLAINER (AI)                  ║
    ║    Schema Compliance:   {sch['compliance_rate']*100:>5.1f}%      ║
    ╚════════════════════════════════════╝
    """
    )
except:
    print("Run the cells above to generate scores.")


    ╔════════════════════════════════════╗
    ║   SALESOPS AGENT REPORT CARD       ║
    ╠════════════════════════════════════╣
    ║ 1. DETECTOR (Math)                 ║
    ║    Recall (Found):      100.0%      ║
    ║    Precision (Noise):     0.1%      ║
    ║    F1 Score:             0.00       ║
    ╠════════════════════════════════════╣
    ║ 2. EXPLAINER (AI)                  ║
    ║    Schema Compliance:   100.0%      ║
    ╚════════════════════════════════════╝
    
